In [2]:
import pickle
dataset_embeddings = pickle.load(open('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/dataset_embeddings.pkl', 'rb'))
index_to_dataset = pickle.load(open('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/index_to_dataset.pkl', 'rb'))
bio_entity_embeddings = pickle.load(open('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/entity_embeddings.pkl', 'rb'))
index_to_bio_entity = pickle.load(open('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/index_to_entity.pkl', 'rb'))

In [3]:
dataset_index_embeddings = {}
for index, dataset in index_to_dataset.items():
    dataset_index_embeddings[index] = dataset_embeddings[dataset]

bio_entity_index_embeddings = {}
for index, entity in index_to_bio_entity.items():
    bio_entity_index_embeddings[index] = bio_entity_embeddings[entity]

In [4]:
import pickle
import gzip

# Load the dictionary from the .gz file
with gzip.open('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/author_embeddings.pkl.gz', 'rb') as f:
    author_embeddings = pickle.load(f)
len(author_embeddings)

44674

In [5]:
author_embeddings = { **author_embeddings, **dataset_index_embeddings, **bio_entity_index_embeddings }

In [6]:
import pandas as pd
authors = pd.read_csv('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/work/data/author_dataset_bio_entity.csv')

In [7]:
author_id_set = set(authors['id'].to_list()) & set(author_embeddings.keys())

In [8]:
authors = authors[authors['id'].isin(author_id_set)]

In [10]:
authors = authors.reset_index(drop=True)
authors['Index'] = authors.index
authors

,id,FullName,BeginYear,PaperNum,CM4AI,RecentYear,Index,is_author,index,color_category,Affiliation,Data_Source,Data_Description,Data_url,pmids_string
0,225,Baba Inusa,2005,102,NaN,2024,0,1,0,0,"Evelina Children's Hospital, Guy's and St. Tho...",NaN,NaN,NaN,"37807937,34339316,35947137,36018224,37171600,3..."
1,878,Rutger Schutten,2017,5,NaN,2022,1,1,1,5,KTH - Royal Institute of Technology,NaN,NaN,NaN,"35931766,33627808,30125267,27723985,28495876"
2,1098,Vardit Ravitsky (CM4AI),2002,192,1.0,2023,2,1,2,3,The Hastings Center,NaN,NaN,NaN,"35725302,36350287,36916763,36919551,37382787,3..."
3,1352,Ursula Grohmann,1988,158,NaN,2023,3,1,3,0,University of Perugia,NaN,NaN,NaN,"35917118,34145969,35409342,33451129,33693650,2..."
4,1592,Sharon R Lewin,1993,373,NaN,2023,4,1,4,5,The University of Melbourne at the Peter Doher...,NaN,NaN,NaN,"36384811,36502576,36541637,36776833,36842873,3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34526,223000004995,type B interruption of the aortic arch,2020,100,0.0,0,34526,0,34526,6,NaN,disease,mim217095,https://bioregistry.io/mim:217095,NaN
34527,223000004996,MPMs,2020,100,0.0,0,34527,0,34527,6,NaN,disease,mim605711,https://bioregistry.io/mim:605711,NaN
34528,223000004997,CRIB,2020,100,0.0,0,34528,0,34528,6,NaN,disease,mim609165,https://bioregistry.io/mim:609165,NaN
34529,223000004998,mesial temporal lobe epilepsy,2020,100,0.0,0,34529,0,34529,6,NaN,disease,mim611630,https://bioregistry.io/mim:611630,NaN


In [11]:
index2aid = authors.groupby('Index')['id'].apply(lambda x: x.iloc[0]).to_dict()
aid2index = {v: k for k, v in index2aid.items()}
author_embeddings_single = {int(k): v for k, v in author_embeddings.items() if k in aid2index}

In [13]:
from tqdm import tqdm
fname = f"/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/work/data/tkg_34kAuthor_Dataset_bioentity.w2v"
with open(fname, 'wt') as f:
    tab = ' '
    nl = '\n'
    f.write(f"{len(author_embeddings_single)}{tab}768{nl}")
    for key, embedding in tqdm(author_embeddings_single.items()):
        f.write(f"{aid2index[key]}{tab}{tab.join(map(str, embedding))}{nl}")
fname

100%|██████████| 34531/34531 [00:10<00:00, 3180.11it/s]


'/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/work/data/tkg_34kAuthor_Dataset_bioentity.w2v'

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import math
from gensim.models import KeyedVectors
import pandas as pd
import emblaze
from emblaze.utils import Field, ProjectionTechnique
from collections import defaultdict, namedtuple

In [15]:
wv = KeyedVectors.load_word2vec_format(fname)

In [16]:
all_author_pd = pd.read_csv('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/work/data/author_dataset_bio_entity.csv')
all_author_pd = all_author_pd[all_author_pd['id'].isin(set(author_embeddings_single.keys()))]

In [ ]:
# all_author_pd.to_csv('/data/jx4237data/projects/web_page/cm4ai_tkg/work/data/author_dataset_filtered.csv', index=False)

In [18]:
all_authors = []
all_author_pd = all_author_pd.reset_index(drop=True)
all_author_pd['index'] = all_author_pd.index
for author in all_author_pd.itertuples(index=False):
    all_authors.append(author)
print(f"author count: {len(all_authors)}; wv len: {len(wv.index_to_key)}")

author count: 34531; wv len: 34531


In [19]:
def ix_to_ix(ix):
    return int(wv.index_to_key[ix])

def author_by_ix(ix):
    ix = ix_to_ix(ix)
    try: 
        all_authors[ix]
    except:
        print(f"ix: {ix}")
    return all_authors[ix]

In [20]:
len(wv.vectors)

34531

In [21]:
positions = wv.vectors
names = [author_by_ix(index).FullName for index in range(len(wv.vectors))]
colors = [author_by_ix(index).BeginYear if int(author_by_ix(index).BeginYear) > 1980 else 1980 for index in range(len(wv.vectors))]
sizes = [math.sqrt(author_by_ix(index).PaperNum)/6.  for index in range(len(wv.vectors))]

emb = emblaze.Embedding({Field.POSITION: positions, Field.NAME: names, Field.COLOR: colors, Field.RADIUS: sizes}, n_neighbors=20,)
# neighbors id are just from 0 to n-1, need wv.index_to_key to get the real index
emb.compute_neighbors(metric='cosine')

In [22]:
import json
import math

projection = emb.project(method=ProjectionTechnique.TSNE, perplexity=40, learning_rate='auto', init='pca', early_exaggeration=1.3)

serialized = projection.to_json(compressed=False, save_neighbors=False)

In [23]:
serialized['ids'] = [author_by_ix(i).id for i in range(len(wv.index_to_key))]

In [24]:
neighbors_json = emb.get_neighbors().to_json(compressed=False)

In [25]:
df_rec_authors = pd.read_csv('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/author_recommendations.csv.gz', compression='gzip')

In [26]:
df_rec_authors['AID_index'] = df_rec_authors['AID'].apply(lambda x: wv.key_to_index.get(str(aid2index.get(x))))
df_rec_authors['recommended_author_id_index'] = df_rec_authors['recommended_author_id'].apply(lambda x: wv.key_to_index.get(str(aid2index.get(x))))
df_rec_authors.dropna(subset=['AID_index', 'recommended_author_id_index'], inplace=True)
df_rec_authors['AID_index'] = df_rec_authors['AID_index'].astype(int)
df_rec_authors['recommended_author_id_index'] = df_rec_authors['recommended_author_id_index'].astype(int)
result_dict = df_rec_authors.groupby('AID_index')['recommended_author_id_index'].apply(list).to_dict()

In [27]:
df_rec_authors_dataset = pd.read_csv('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/dataset_author_recommendations.csv.gz', compression='gzip')

df_rec_authors_dataset['DatasetID_index'] = df_rec_authors_dataset['DatasetID'].apply(lambda x: wv.key_to_index.get(str(aid2index.get(x))))
df_rec_authors_dataset['recommended_author_id_index'] = df_rec_authors_dataset['recommended_author_id'].apply(lambda x: wv.key_to_index.get(str(aid2index.get(x))))
df_rec_authors_dataset.dropna(subset=['DatasetID_index', 'recommended_author_id_index'], inplace=True)
df_rec_authors_dataset['DatasetID_index'] = df_rec_authors_dataset['DatasetID_index'].astype(int)
df_rec_authors_dataset['recommended_author_id_index'] = df_rec_authors_dataset['recommended_author_id_index'].astype(int)
result_dict_dataset = df_rec_authors_dataset.groupby('DatasetID_index')['recommended_author_id_index'].apply(list).to_dict()

In [28]:
df_rec_authors_bio = pd.read_csv('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/raw_data/bioentity_author_recommendations.csv.gz', compression='gzip')
df_rec_authors_bio['BioEntityID_index'] = df_rec_authors_bio['BioID'].apply(lambda x: wv.key_to_index.get(str(aid2index.get(x))))
df_rec_authors_bio['recommended_author_id_index'] = df_rec_authors_bio['Recommended AID'].apply(lambda x: wv.key_to_index.get(str(aid2index.get(x))))
df_rec_authors_bio.dropna(subset=['BioEntityID_index', 'recommended_author_id_index'], inplace=True)
df_rec_authors_bio['BioEntityID_index'] = df_rec_authors_bio['BioEntityID_index'].astype(int)
df_rec_authors_bio['recommended_author_id_index'] = df_rec_authors_bio['recommended_author_id_index'].astype(int)
result_dict_bio = df_rec_authors_bio.groupby('BioEntityID_index')['recommended_author_id_index'].apply(list).to_dict()

In [29]:
result_dict = {**result_dict, **result_dict_dataset, **result_dict_bio}

In [30]:
neighbors_json['neighbors'] = result_dict

In [31]:
serialized['neighbors'] = neighbors_json
with open('/home/jx4237/projects/web_page/knowledge_graph_vis/knowledge_graph_viz/work/data/tkg_ebd_34k_dataset_bioentity.json', 'wt') as f:
    f.write(json.dumps(serialized))